In [ ]:
## Version 1:
## int 32 for user id
## Simple Mean of answered Correctly for Each  User Id

Description :
1. In this notebook I have tried First Submission of My Competition .
2. Rather than simply Taking Mean of Content Ids , I am trying to take mean of User Ids and Based on their number of correct answers we are predicting ! 
3. This is obiviously not a good solution as it causes target leakage describe [here](https://www.kaggle.com/c/riiid-test-answer-prediction/discussion/189437)
4. More to add is Rolling Average and Bayesian Averages ! 

In [ ]:
import riiideducation
import dask.dataframe as dd

env = riiideducation.make_env()

In [ ]:
user_acc = dd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                usecols=[2,7], dtype={'user_id': 'int32','answered_correctly':'int8'}
              ).query('answered_correctly != -1').groupby('user_id')['answered_correctly'].mean().compute().to_dict()


In [ ]:
def add_user_acc(x):
    if x in user_acc.keys():
        return user_acc[x]
    else:
        return 0.5

iter_test = env.iter_test()
batch = 0 
for (test_df, sample_prediction_df) in iter_test:
    test_df['answered_correctly'] = test_df['user_id'].apply(add_user_acc).values
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])
    print("Batch Done : " , batch)
    batch += 1